<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Complete LangChain Guide

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

Comprehensive LangChain integration with Gemini 3 Pro.

In [ ]:
!pip install -q langchain langchain-google-genai faiss-cpu

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## 1. Prompt Templates

In [ ]:
# Simple template
llm = ChatGoogleGenerativeAI(model="gemini-3-pro", temperature=0.7)

template = PromptTemplate(
    input_variables=["topic"],
    template="Write a short paragraph about {topic}."
)

chain = LLMChain(llm=llm, prompt=template)
result = chain.run(topic="artificial intelligence")
print(result)

In [ ]:
# Few-shot template
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "hot", "output": "cold"},
]

example_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="Give the opposite of the word.",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"]
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt)
print(chain.run(input="bright"))

## 2. Chains

In [ ]:
# Sequential chain
synopsis_template = PromptTemplate(
    input_variables=["title"],
    template="Write a synopsis for a movie titled '{title}'."
)

review_template = PromptTemplate(
    input_variables=["synopsis"],
    template="Write a review for this movie synopsis:\n{synopsis}"
)

synopsis_chain = LLMChain(llm=llm, prompt=synopsis_template, output_key="synopsis")
review_chain = LLMChain(llm=llm, prompt=review_template, output_key="review")

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["title"],
    output_variables=["synopsis", "review"]
)

result = overall_chain({"title": "AI Revolution"})
print(f"Synopsis: {result['synopsis']}\n")
print(f"Review: {result['review']}")

## 3. Memory

In [ ]:
# Conversation with memory
memory = ConversationBufferMemory()

conversation_template = PromptTemplate(
    input_variables=["history", "input"],
    template="""
Previous conversation:
{history}

Human: {input}
AI:
"""
)

conversation_chain = LLMChain(
    llm=llm,
    prompt=conversation_template,
    memory=memory
)

print(conversation_chain.run(input="Hi, I'm learning about AI."))
print("\n" + conversation_chain.run(input="What did I just say?"))

## 4. Document Processing & RAG

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Sample documents
docs = [
    Document(page_content="LangChain is a framework for developing LLM applications."),
    Document(page_content="It provides tools for prompts, chains, and agents."),
    Document(page_content="LangChain supports multiple LLM providers including Gemini.")
]

# Create vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(docs, embeddings)

# QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

result = qa_chain({"query": "What is LangChain?"})
print(f"Answer: {result['result']}")

## 5. Agents

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [get_word_length, multiply]
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

result = agent_executor.invoke({
    "input": "How many letters in 'LangChain' times 5?"
})
print(result["output"])

## 6. LCEL (LangChain Expression Language)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# LCEL chain
prompt = ChatPromptTemplate.from_template(
    "Tell me a {adjective} fact about {topic}"
)

chain = (
    {"adjective": RunnablePassthrough(), "topic": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke({"adjective": "interesting", "topic": "AI"})
print(result)

## Key Takeaways

✅ **LangChain Components:**
- **Prompts**: Templates and few-shot examples
- **Chains**: Sequential and complex workflows
- **Memory**: Conversation history
- **Agents**: Autonomous decision-making
- **LCEL**: Composable chains

📌 **Best Practices:**
- Use LCEL for modern chains
- Leverage memory for conversations
- Build modular components
- Test chains thoroughly

🔗 **Resources:**
- [LangChain Docs](https://python.langchain.com/)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI)